In [28]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score

In [29]:
train_dataset = pd.read_csv("train.csv")
test_dataset = pd.read_csv("test.csv")
train_dataset.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [30]:
print(train_dataset.isnull().sum().sort_values())
print("\n\n")
print(train_dataset.isna().sum().sort_values())


print(test_dataset.isnull().sum().sort_values())
print("\n\n")
print(test_dataset.isna().sum().sort_values())

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
SibSp            0
Parch            0
Ticket           0
Fare             0
Embarked         2
Age            177
Cabin          687
dtype: int64



PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
SibSp            0
Parch            0
Ticket           0
Fare             0
Embarked         2
Age            177
Cabin          687
dtype: int64
PassengerId      0
Pclass           0
Name             0
Sex              0
SibSp            0
Parch            0
Ticket           0
Embarked         0
Fare             1
Age             86
Cabin          327
dtype: int64



PassengerId      0
Pclass           0
Name             0
Sex              0
SibSp            0
Parch            0
Ticket           0
Embarked         0
Fare             1
Age             86
Cabin          327
dtype: int64


In [31]:
train_dataset_encoded = pd.get_dummies(data=train_dataset, columns=["Sex", "Pclass", "Embarked"], dtype=int)
test_dataset_encoded = pd.get_dummies(data=test_dataset, columns=["Sex", "Pclass", "Embarked"], dtype=int)

In [34]:
test_passenger_ids = test_dataset_encoded["PassengerId"].values

In [5]:
train_dataset_encoded.drop(columns=["PassengerId", "Ticket", "Name", "Cabin"], axis=1, inplace=True)
test_dataset_encoded.drop(columns=["PassengerId", "Ticket", "Name", "Cabin"], axis=1, inplace=True)

In [6]:
print(len(train_dataset) * 0.22)
print(len(train_dataset) - (len(train_dataset) * 0.22))
print(len(test_dataset))
print(f"{train_dataset['Survived'].count()}")
print(f"Count of Succumbed: {len(train_dataset[train_dataset['Survived'] == 0])}")
print(f"Count of Survived: {len(train_dataset[train_dataset['Survived'] == 1])}")

196.02
694.98
418
891
Count of Succumbed: 549
Count of Survived: 342


In [23]:
imputer = IterativeImputer(estimator=RandomForestRegressor(), random_state=10)
train_dataset_encoded["Age"] = imputer.fit_transform(train_dataset_encoded[["Age"]])
test_dataset_encoded["Age"] = imputer.fit_transform(test_dataset_encoded[["Age"]])
test_dataset_encoded["Fare"] = imputer.fit_transform(test_dataset_encoded[["Fare"]])

In [24]:
X_train = train_dataset_encoded.drop(columns="Survived", axis=1)
y_train = train_dataset_encoded["Survived"]

In [15]:
training_features = X_train.columns
len(training_features)

12

In [25]:
test_dataset_encoded.isnull().sum()

Age           0
SibSp         0
Parch         0
Fare          0
Sex_female    0
Sex_male      0
Pclass_1      0
Pclass_2      0
Pclass_3      0
Embarked_C    0
Embarked_Q    0
Embarked_S    0
dtype: int64

In [26]:
test_dataset_encoded

,Age,SibSp,Parch,Fare,Sex_female,Sex_male,Pclass_1,Pclass_2,Pclass_3,Embarked_C,Embarked_Q,Embarked_S
0,34.50000,0,0,7.8292,0,1,0,0,1,0,1,0
1,47.00000,1,0,7.0000,1,0,0,0,1,0,0,1
2,62.00000,0,0,9.6875,0,1,0,1,0,0,1,0
3,27.00000,0,0,8.6625,0,1,0,0,1,0,0,1
4,22.00000,1,1,12.2875,1,0,0,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...
413,30.27259,0,0,8.0500,0,1,0,0,1,0,0,1
414,39.00000,0,0,108.9000,1,0,1,0,0,1,0,0
415,38.50000,0,0,7.2500,0,1,0,0,1,0,0,1
416,30.27259,0,0,8.0500,0,1,0,0,1,0,0,1
